# Circuit Library - Code Laboratory

**Section 3: Create Circuits - Part 4** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Algorithm Circuits
| Class/Function | Signature | Returns | Use When |
|----------------|-----------|---------|----------|
| `QFTGate` | `QFTGate(num_qubits)` | `Gate` | Quantum Fourier Transform |
| `.inverse()` | `gate.inverse()` | `Gate` | Inverse of any gate |

### VQE Ansätze (⚠️ EXAM CRITICAL!)
| Function | Signature | Rotations | Parameters |
|----------|-----------|-----------|------------|
| `real_amplitudes()` | `real_amplitudes(num_qubits, reps=...)` | RY only | `n × (reps+1)` |
| `efficient_su2()` | `efficient_su2(num_qubits, reps=...)` | RY + RZ | `2n × (reps+1)` |
| `n_local()` | `n_local(num_qubits, rotation_blocks, entanglement_blocks, reps)` | Custom | Varies |

### Comparison
| Ansatz | Parameters (n=3, reps=2) | Expressivity |
|--------|--------------------------|--------------|
| `real_amplitudes` | 9 | Lower |
| `efficient_su2` | 18 | Higher |

---

In [ ]:
"""
Qiskit Code Laboratory - Circuit Library
=========================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    QFTGate,              # Quantum Fourier Transform
    real_amplitudes,      # VQE ansatz (RY only)
    efficient_su2,        # VQE ansatz (RY + RZ)
    n_local,              # Custom ansatz builder
    XGate, YGate, ZGate, HGate, CXGate  # Standard gates
)
from qiskit.quantum_info import Statevector, Operator

# Utility functions for this notebook
def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_ansatz_info(ansatz, name="Ansatz"):
    """Display ansatz information."""
    print(f"{name}:")
    print(f"  Qubits: {ansatz.num_qubits}")
    print(f"  Parameters: {ansatz.num_parameters}")
    print(f"  Depth: {ansatz.depth()}")

print("✅ Environment ready - Circuit Library Lab")

---

## `QFTGate` - Quantum Fourier Transform

### Signature
```python
QFTGate(num_qubits: int) -> Gate
```

### Parameters
| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `num_qubits` | `int` | Yes | Number of qubits |

### Returns
`Gate` - QFT gate to append to circuit

### Key Methods
- `.inverse()` - Get inverse QFT

---

### Basic Usage

In [ ]:
# ============================================================
# QFTGate - BASIC USAGE
# ============================================================

# Create QFT gate
qft3 = QFTGate(3)

# Add to circuit
qc = QuantumCircuit(3)
qc.append(qft3, range(3))

print("QFT on 3 qubits:")
print(qc.draw())

# Verify properties
assert qft3.num_qubits == 3, "QFT should have 3 qubits"
print(f"\n✅ QFT gate created with {qft3.num_qubits} qubits")

In [ ]:
# ============================================================
# QFTGate - INVERSE QFT
# ============================================================

qft3 = QFTGate(3)
iqft3 = QFTGate(3).inverse()

# QFT followed by inverse = identity
qc = QuantumCircuit(3)
qc.h(0)  # Prepare non-trivial state
qc.append(qft3, range(3))
qc.append(iqft3, range(3))

print("QFT + Inverse QFT:")
print(qc.draw())

# Verify: QFT · IQFT = Identity
sv_before = Statevector.from_instruction(QuantumCircuit(3).compose(QuantumCircuit(3, name='init').compose(QuantumCircuit(3))))
sv = Statevector.from_instruction(qc)

# Just H(0) without QFT·IQFT
qc_ref = QuantumCircuit(3)
qc_ref.h(0)
sv_ref = Statevector.from_instruction(qc_ref)

assert np.allclose(sv.data, sv_ref.data), "QFT·IQFT should be identity"
print("\n✅ Verified: QFT · IQFT = Identity")

In [ ]:
# ============================================================
# QFTGate - ALGORITHM USAGE
# ============================================================

# QFT in larger algorithm (QPE pattern)
qc = QuantumCircuit(3)
qc.h(range(3))           # Superposition
qc.append(QFTGate(3), range(3))  # QFT

print("QFT in algorithm pattern:")
print(qc.draw())

print("\n📌 EXAM: QFT used in:")
print("   - Shor's algorithm (period finding)")
print("   - Quantum Phase Estimation (QPE)")
print("   - HHL algorithm")

---

## `real_amplitudes()` - VQE Ansatz (⚠️ EXAM CRITICAL!)

### Signature
```python
real_amplitudes(num_qubits, entanglement='reverse_linear', reps=3, ...) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `num_qubits` | `int` | Required | Number of qubits |
| `reps` | `int` | `3` | Number of repetition blocks |
| `entanglement` | `str` | `'reverse_linear'` | 'linear', 'full', 'circular' |

### Returns
`QuantumCircuit` - Parameterized ansatz

### Parameter Count Formula
`num_qubits × (reps + 1)`

---

### Basic Usage

In [ ]:
# ============================================================
# real_amplitudes() - BASIC USAGE
# ============================================================

ansatz = real_amplitudes(num_qubits=3, reps=2)

print("RealAmplitudes(3 qubits, 2 reps):")
print(ansatz.draw())

# Verify parameter formula: n × (reps + 1)
expected_params = 3 * (2 + 1)  # 3 × 3 = 9
assert ansatz.num_parameters == expected_params, f"Expected {expected_params} params"

print(f"\nParameters: {ansatz.num_parameters}")
print(f"Formula: num_qubits × (reps + 1) = 3 × 3 = 9 ✅")
print(f"Depth: {ansatz.depth()}")

---

### Parameter Variations: reps Effect

In [ ]:
# ============================================================
# real_amplitudes() - REPS EFFECT
# ============================================================

print("Effect of reps on RealAmplitudes (2 qubits):")
print("="*50)

for reps in [1, 2, 3]:
    ansatz = real_amplitudes(num_qubits=2, reps=reps)
    expected = 2 * (reps + 1)
    assert ansatz.num_parameters == expected, f"Expected {expected}"
    print(f"reps={reps}: {ansatz.num_parameters} params, depth={ansatz.depth()}")
    print(f"       Formula: 2 × ({reps}+1) = {expected} ✅")

print("\n✅ More reps = more parameters = more expressivity!")

In [ ]:
# ============================================================
# real_amplitudes() - ENTANGLEMENT PATTERNS
# ============================================================

print("Entanglement Patterns:")
print("="*50)

print("\nLinear entanglement:")
ansatz_linear = real_amplitudes(3, entanglement='linear', reps=1)
print(ansatz_linear.draw())

print("\nFull entanglement:")
ansatz_full = real_amplitudes(3, entanglement='full', reps=1)
print(ansatz_full.draw())

print("\n📌 EXAM: Know entanglement patterns!")
print("   linear: CX chain (0→1→2)")
print("   full: All pairs connected")

In [ ]:
# ============================================================
# real_amplitudes() - BINDING (VQE Pattern)
# ============================================================

ansatz = real_amplitudes(2, reps=1)
print("Before binding:")
print(ansatz.draw())
print(f"Parameters: {ansatz.num_parameters}")

# Bind with values
param_values = np.random.random(ansatz.num_parameters) * 2 * np.pi
bound_ansatz = ansatz.assign_parameters(param_values)

print("\nAfter binding:")
print(bound_ansatz.draw())

# Verify binding
assert bound_ansatz.num_parameters == 0, "All should be bound"
print(f"\n✅ Parameters remaining: {bound_ansatz.num_parameters}")

---

## `efficient_su2()` - More Expressive Ansatz

### Signature
```python
efficient_su2(num_qubits, reps=3, entanglement='reverse_linear', ...) -> QuantumCircuit
```

### Key Difference from RealAmplitudes
- Uses **both RY and RZ** rotations
- **2× more parameters** than RealAmplitudes
- More expressivity for complex problems

### Parameter Count Formula
`2 × num_qubits × (reps + 1)`

---

In [ ]:
# ============================================================
# efficient_su2() - BASIC USAGE
# ============================================================

ansatz = efficient_su2(num_qubits=3, reps=2)

print("EfficientSU2(3 qubits, 2 reps):")
print(ansatz.draw())

# Verify parameter formula: 2 × n × (reps + 1)
expected_params = 2 * 3 * (2 + 1)  # 2 × 3 × 3 = 18
assert ansatz.num_parameters == expected_params, f"Expected {expected_params} params"

print(f"\nParameters: {ansatz.num_parameters}")
print(f"Formula: 2 × num_qubits × (reps + 1) = 2 × 3 × 3 = 18 ✅")

In [ ]:
# ============================================================
# COMPARISON: RealAmplitudes vs EfficientSU2
# ============================================================

print("="*60)
print("COMPARISON: RealAmplitudes vs EfficientSU2")
print("="*60)

n_qubits = 3
reps = 2

real_amp = real_amplitudes(n_qubits, reps=reps)
eff_su2 = efficient_su2(n_qubits, reps=reps)

print(f"\nFor {n_qubits} qubits, {reps} reps:")
print(f"  RealAmplitudes: {real_amp.num_parameters} params (RY only)")
print(f"  EfficientSU2:   {eff_su2.num_parameters} params (RY + RZ)")

# Verify ratio
assert eff_su2.num_parameters == 2 * real_amp.num_parameters
print(f"\n✅ EfficientSU2 has 2× more parameters!")

---

## `n_local()` - Custom Ansatz Builder

### Signature
```python
n_local(num_qubits, rotation_blocks, entanglement_blocks, reps=3, ...) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Description |
|-----------|------|-------------|
| `rotation_blocks` | `str` or `list` | Single-qubit gates: 'ry', 'rz', ['ry', 'rz'] |
| `entanglement_blocks` | `str` | Two-qubit gates: 'cx', 'cz', 'swap' |
| `reps` | `int` | Repetitions |

---

In [ ]:
# ============================================================
# n_local() - CUSTOM ANSATZ
# ============================================================

# RY rotations + CZ entangling
ansatz = n_local(
    num_qubits=3,
    rotation_blocks='ry',
    entanglement_blocks='cz',
    reps=2
)

print("n_local(RY + CZ):")
print(ansatz.draw())
print(f"\nParameters: {ansatz.num_parameters}")

In [ ]:
# ============================================================
# n_local() - MULTIPLE ROTATION BLOCKS
# ============================================================

# Both RY and RZ rotations
ansatz = n_local(
    num_qubits=2,
    rotation_blocks=['ry', 'rz'],  # Multiple rotations
    entanglement_blocks='cx',
    reps=1
)

print("n_local([RY, RZ] + CX):")
print(ansatz.draw())
print(f"\nParameters: {ansatz.num_parameters}")
print("\n✅ n_local gives full control over ansatz structure!")

---

## Gate Objects - Using Library Gates

---

In [ ]:
# ============================================================
# GATE OBJECTS - Using .append()
# ============================================================

qc = QuantumCircuit(2)

# Append gate objects
qc.append(HGate(), [0])
qc.append(XGate(), [1])
qc.append(CXGate(), [0, 1])

print("Circuit with gate objects:")
print(qc.draw())

# Verify operations
ops = qc.count_ops()
assert 'h' in ops and 'x' in ops and 'cx' in ops
print("\n✅ Gates can be used as objects with .append()!")

---

## 🏋️ Code Challenges

> These are CODING challenges - solve them by writing code.
> For conceptual questions, see [README.md](./README.md) practice section.

---

In [ ]:
# ============================================================
# CHALLENGE 1: Parameter Count
# ============================================================
# Task: Create RealAmplitudes with EXACTLY 12 parameters
# Constraint: Use 4 qubits
# ============================================================

def challenge_1():
    """
    Create a RealAmplitudes ansatz with:
    - num_qubits = 4
    - Exactly 12 parameters
    
    Hint: params = num_qubits × (reps + 1)
          12 = 4 × (reps + 1)
          reps = ?
    
    Returns:
        QuantumCircuit: Ansatz with 12 parameters
    """
    # YOUR CODE HERE
    # What reps value gives 12 params for 4 qubits?
    ansatz = real_amplitudes(4, reps=???)  # Replace ???
    return ansatz

# Uncomment to test:
# ansatz = challenge_1()
# print(f"Parameters: {ansatz.num_parameters}")
# if ansatz.num_parameters == 12:
#     print("✅ Challenge 1 PASSED!")
# else:
#     print("❌ Challenge 1 FAILED")

In [ ]:
# ============================================================
# SOLUTION: Challenge 1
# ============================================================

def challenge_1_solution():
    """
    12 = 4 × (reps + 1)
    3 = reps + 1
    reps = 2
    """
    ansatz = real_amplitudes(4, reps=2)
    return ansatz

ansatz_sol = challenge_1_solution()
print(f"Parameters: {ansatz_sol.num_parameters}")
print(f"✅ 4 × (2 + 1) = 12")

In [ ]:
# ============================================================
# CHALLENGE 2: EfficientSU2 vs RealAmplitudes (EXAM CRITICAL!)
# ============================================================
# Task: Understand the parameter difference between ansätze
# Exam question: "How many params does EfficientSU2(4, reps=2) have?"
# ============================================================

def challenge_2():
    """
    Compare parameter counts:
    - RealAmplitudes: num_qubits × (reps + 1)
    - EfficientSU2: 2 × num_qubits × (reps + 1)
    
    Task: For 4 qubits, 2 reps:
    - RealAmplitudes should have: 4 × 3 = 12 params
    - EfficientSU2 should have: 2 × 4 × 3 = 24 params
    """
    n_qubits = 4
    reps = 2
    
    ra = real_amplitudes(n_qubits, reps=reps)
    es = efficient_su2(n_qubits, reps=reps)
    
    print(f"RealAmplitudes({n_qubits}, reps={reps})")
    print(f"  Formula: n × (reps + 1) = {n_qubits} × {reps+1} = {n_qubits * (reps + 1)}")
    print(f"  Actual: {ra.num_parameters} parameters")
    
    print(f"\nEfficientSU2({n_qubits}, reps={reps})")
    print(f"  Formula: 2 × n × (reps + 1) = 2 × {n_qubits} × {reps+1} = {2 * n_qubits * (reps + 1)}")
    print(f"  Actual: {es.num_parameters} parameters")
    
    # Verify
    assert ra.num_parameters == n_qubits * (reps + 1), "RealAmplitudes formula"
    assert es.num_parameters == 2 * n_qubits * (reps + 1), "EfficientSU2 formula"
    
    print(f"\n✅ Challenge 2 PASSED!")
    print(f"   EfficientSU2 has {es.num_parameters // ra.num_parameters}× more params than RealAmplitudes")
    
    return ra, es

# Run
ra, es = challenge_2()

---

## PhaseOracle - Boolean Functions for Grover's (EXAM!)

PhaseOracle creates oracles that flip the phase of target states - essential for Grover's algorithm.

In [ ]:
# ============================================================
# PhaseOracle - From Boolean Expression (GROVER'S!)
# ============================================================

from qiskit.circuit.library import PhaseOracle

# Oracle from Boolean expression
# This marks |10⟩ and |11⟩ as solutions (states where first bit is 1)
oracle = PhaseOracle('x0')  # x0 = True means qubit 0 is 1

print("PhaseOracle('x0') - Marks states where q0=1:")
print(oracle.draw())

# Verify: Should flip phase of |10⟩ and |11⟩
from qiskit.quantum_info import Statevector
test_qc = QuantumCircuit(2)
test_qc.h([0, 1])  # Equal superposition
test_qc.compose(oracle, inplace=True)

sv = Statevector.from_instruction(test_qc)
print(f"\nState after oracle: amplitudes have phase flips")
print(f"✅ PhaseOracle creates Grover oracles from Boolean expressions")

In [ ]:
# ============================================================
# PhaseOracle - Complex Boolean Expressions
# ============================================================

# AND expression: Both qubits must be 1 → marks |11⟩
oracle_and = PhaseOracle('x0 & x1')  # x0 AND x1
print("PhaseOracle('x0 & x1') - Marks only |11⟩:")
print(oracle_and.draw())

# OR expression: Either qubit is 1 → marks |01⟩, |10⟩, |11⟩
oracle_or = PhaseOracle('x0 | x1')  # x0 OR x1
print("\nPhaseOracle('x0 | x1') - Marks |01⟩, |10⟩, |11⟩:")
print(oracle_or.draw())

# XOR expression: Exactly one qubit is 1 → marks |01⟩, |10⟩
oracle_xor = PhaseOracle('x0 ^ x1')  # x0 XOR x1
print("\nPhaseOracle('x0 ^ x1') - Marks |01⟩, |10⟩:")
print(oracle_xor.draw())

print("\n📌 EXAM: Boolean operators in PhaseOracle:")
print("   & = AND, | = OR, ^ = XOR, ~ = NOT")

---

## 📚 Resources

### Code References
- [Qiskit Circuit Library](https://docs.quantum.ibm.com/api/qiskit/circuit_library)
- [QFTGate API](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.QFTGate)

### Related Materials
- [README.md](./README.md) - Conceptual explanations, traps, mnemonics
- [Practice Questions](./README.md#practice-exam) - Text-based exam practice

### Next Notebook
- [Classical Control](./classical_control.ipynb) - Conditional operations

---

### Quick Reference Card

```
┌─────────────────────────────────────────────────────────────────┐
│ CIRCUIT LIBRARY - QUICK REFERENCE                               │
├─────────────────────────────────────────────────────────────────┤
│ QFT:                                                            │
│   QFTGate(n)           - n-qubit QFT                           │
│   QFTGate(n).inverse() - Inverse QFT                           │
├─────────────────────────────────────────────────────────────────┤
│ VQE ANSÄTZE:                                                    │
│   real_amplitudes(n, reps=k)  - RY only, n×(k+1) params        │
│   efficient_su2(n, reps=k)    - RY+RZ, 2n×(k+1) params         │
│   n_local(n, rot, ent, reps)  - Custom ansatz                  │
├─────────────────────────────────────────────────────────────────┤
│ PARAMETER FORMULAS:                                             │
│   RealAmplitudes: num_qubits × (reps + 1)                      │
│   EfficientSU2:   2 × num_qubits × (reps + 1)                  │
├─────────────────────────────────────────────────────────────────┤
│ EXAM FACTS:                                                     │
│   QFT used in: Shor's, QPE, HHL                                │
│   EfficientSU2 has 2× params of RealAmplitudes                 │
│   n_local is the general template                              │
└─────────────────────────────────────────────────────────────────┘
```

---

**Notebook verified with Qiskit 1.x** | Last updated: 2025